In [1]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [112]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [113]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [137]:
freq = 4
#:Price_10Y
returns = percentchange(data_source[:Total_Stock], :log)
#returns = merge(returns, log1p.(data_source[:T_Bill])./freq)
cpi = percentchange(data_source[:CPI], :log)
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
term = data_source[:Yeld_10Y] ./freq .- data_source[:T_Bill] ./freq
dy = log.(data_source[:Dividend_Yeld])
data = merge(excess_returns, real_t_bill, term, cpi)

291×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬────────────────────┬─────────────┬─────────────────┬─────────────
│            │ Total_Stock_T_Bill │ T_Bill_CPI  │ Yeld_10Y_T_Bill │ CPI        ⋯
├────────────┼────────────────────┼─────────────┼─────────────────┼─────────────
│ 1952-03-01 │        -0.00772801 │  0.00772445 │       0.0022875 │ -0.0037807 ⋯
│ 1952-06-01 │          0.0651996 │ -0.00708528 │       0.0017875 │   0.011299 ⋯
│ 1952-09-01 │         -0.0227685 │  0.00423886 │       0.0024375 │         0. ⋯
│ 1952-12-01 │            0.08432 │   0.0089235 │       0.0011425 │ -0.0037523 ⋯
│ 1953-03-01 │         -0.0487217 │  0.00497517 │       0.0018125 │         0. ⋯
│ 1953-06-01 │        -0.00778441 │ -0.00227055 │       0.0021925 │  0.0074906 ⋯
│ 1953-09-01 │        -0.00277599 │ -0.00299956 │        0.002115 │  0.0074349 ⋯
│ 1953-12-01 │          0.0705234 │  0.00767892 │       0.0021725 │ -0.0037105 ⋯
│     ⋮      │         ⋮         

In [138]:
CSV.write("usa_data_transformed.csv", data)

"usa_data_transformed.csv"

In [161]:
start = Date(2001,01,01)
end_d = Date(2019, 12, 31)
s_data = to(from(data, start),end_d)

returns_summarystats(to(from(returns, start),end_d), freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1409,0.5492,0.0665,0.247,-1.0147,-1.3194,-0.0543,0.2051,2.4139,0.4721,0.1428


In [162]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.071,0.137,0.0132,0.0616,-0.2565,-1.2895,-0.015,0.2125,2.3009,0.1857,0.0327
T_Bill_CPI,0.0085,0.0255,-0.0017,0.0027,-0.0208,0.5919,-0.0071,-0.0202,0.9867,-0.1993,-0.0024
Yeld_10Y_T_Bill,0.0027,0.0095,0.0047,0.0066,-0.0007,-0.2828,0.0026,0.8665,-0.8113,1.7287,0.0049
CPI,0.0084,0.0236,0.0051,0.0099,-0.0254,-0.5828,0.0011,-0.0763,1.8055,0.6078,0.0046


In [163]:
returns_summarystats(s_data, freq)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_T_Bill,0.142,0.548,0.0527,0.2465,-1.026,-1.2895,-0.06,0.2125,2.3009,0.3713,0.1309
T_Bill_CPI,0.017,0.1019,-0.0068,0.0109,-0.0832,0.5919,-0.0282,-0.0202,0.9867,-0.3986,-0.0097
Yeld_10Y_T_Bill,0.0054,0.038,0.0187,0.0265,-0.0029,-0.2828,0.0104,0.8665,-0.8113,3.4575,0.0198
CPI,0.0168,0.0946,0.0204,0.0394,-0.1016,-0.5828,0.0045,-0.0763,1.8055,1.2155,0.0183


In [164]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI,std
Total_Stock_T_Bill,0.0889,0.172,-7.4265,-9.8461,-8.2519,0.065
T_Bill_CPI,-0.0026,-0.0424,0.4028,-0.1764,0.5763,0.0077
Yeld_10Y_T_Bill,0.0006,-0.0,-0.021,0.8712,-0.012,0.0013
CPI,0.0027,0.0478,0.5499,0.1653,0.3721,0.0079


,Total_Stock_T_Bill,T_Bill_CPI,Yeld_10Y_T_Bill,CPI
Total_Stock_T_Bill,1.0,-0.11,0.15,0.12
T_Bill_CPI,-0.11,1.0,-0.16,-0.99
Yeld_10Y_T_Bill,0.15,-0.16,1.0,0.1
CPI,0.12,-0.99,0.1,1.0


In [165]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

"Total_Stock_T_Bill coefficients"

Quantiles
          parameters       2.5%      25.0%     50.0%     75.0%     97.5% 
              Symbol    Float64    Float64   Float64   Float64   Float64 

               const     0.0339     0.0707    0.0891    0.1074    0.1436
  Total_Stock_T_Bill    -0.0541     0.0964    0.1718    0.2470    0.3945
          T_Bill_CPI   -13.2819    -9.4171   -7.4334   -5.4468   -1.4674
     Yeld_10Y_T_Bill   -17.6282   -12.4913   -9.8592   -7.2559   -2.0423
                 CPI   -14.0206   -10.2365   -8.2741   -6.3258   -2.4636


"T_Bill_CPI coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0090   -0.0048   -0.0026   -0.0003    0.0039
  Total_Stock_T_Bill   -0.0697   -0.0514   -0.0426   -0.0333   -0.0153
          T_Bill_CPI   -0.3212    0.1574    0.3958    0.6428    1.1028
     Yeld_10Y_T_Bill   -1.1025   -0.4987   -0.1782    0.1352    0.7415
                 CPI   -0.1177    0.3364    0.5705    0.8079    1.2626


"Yeld_10Y_T_Bill coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0004    0.0003    0.0006    0.0010    0.0017
  Total_Stock_T_Bill   -0.0044   -0.0015   -0.0000    0.0015    0.0044
          T_Bill_CPI   -0.1376   -0.0610   -0.0208    0.0193    0.0934
     Yeld_10Y_T_Bill    0.7184    0.8187    0.8716    0.9248    1.0235
                 CPI   -0.1253   -0.0509   -0.0123    0.0269    0.0995


"CPI coefficients"

Quantiles
          parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
              Symbol   Float64   Float64   Float64   Float64   Float64 

               const   -0.0039    0.0004    0.0027    0.0049    0.0092
  Total_Stock_T_Bill    0.0200    0.0386    0.0479    0.0570    0.0753
          T_Bill_CPI   -0.1662    0.3041    0.5541    0.7945    1.2805
     Yeld_10Y_T_Bill   -0.7609   -0.1502    0.1658    0.4965    1.0990
                 CPI   -0.3252    0.1372    0.3753    0.6159    1.0788


"correlation matrix"

Quantiles
                             parameters      2.5%     25.0%     50.0%     75.0 ⋯
                                 Symbol   Float64   Float64   Float64   Float6 ⋯

  Total_Stock_T_Bill_Total_Stock_T_Bill    0.0569    0.0631    0.0668    0.070 ⋯
          Total_Stock_T_Bill_T_Bill_CPI   -0.3403   -0.1940   -0.1119   -0.032 ⋯
     Total_Stock_T_Bill_Yeld_10Y_T_Bill   -0.0841    0.0742    0.1554    0.233 ⋯
                 Total_Stock_T_Bill_CPI   -0.1135    0.0421    0.1225    0.202 ⋯
          T_Bill_CPI_Total_Stock_T_Bill   -0.3403   -0.1940   -0.1119   -0.032 ⋯
                  T_Bill_CPI_T_Bill_CPI    0.0068    0.0075    0.0079    0.008 ⋯
             T_Bill_CPI_Yeld_10Y_T_Bill   -0.3852   -0.2461   -0.1666   -0.083 ⋯
                         T_Bill_CPI_CPI   -0.9960   -0.9946   -0.9936   -0.992 ⋯
     Yeld_10Y_T_Bill_Total_Stock_T_Bill   -0.0841    0.0742    0.1554    0.233 ⋯
             Yeld_10Y_T_Bill_T_Bill_CPI   -0.3852   -0.2461   -0.1666   -0.083 ⋯
        Yeld_10Y_

In [166]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel, freq * 15)

for n in 1:length(assets_names)
   print_percentiles(scenarios[n, :, 2:end], [0.02, 0.05, 0.25, 0.5, 0.75, 0.95, 0.98], freq, string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0979,-0.2074,-0.2422,-0.2565,-0.2627,-0.2618,-0.2516,-0.2586,-0.2499,-0.2496,-0.2469,-0.2481,-0.2473,-0.2472,-0.2557
0.05,-0.0411,-0.149,-0.1825,-0.1948,-0.1946,-0.1916,-0.19,-0.1908,-0.1852,-0.1795,-0.1799,-0.1829,-0.1833,-0.1801,-0.1869
0.25,0.0932,0.0049,-0.0276,-0.0342,-0.0364,-0.0361,-0.0335,-0.0322,-0.0272,-0.0249,-0.0264,-0.0272,-0.0258,-0.0238,-0.0214
0.5,0.1881,0.1107,0.0844,0.0746,0.0736,0.0775,0.0797,0.08,0.0804,0.0854,0.0877,0.0829,0.0846,0.089,0.0892
0.75,0.2835,0.2153,0.191,0.1865,0.1828,0.1873,0.1849,0.1904,0.193,0.1928,0.1995,0.1927,0.1939,0.1967,0.2018
0.95,0.4204,0.3686,0.3476,0.3408,0.3387,0.3388,0.3405,0.3423,0.3494,0.3474,0.3616,0.3515,0.3561,0.3566,0.3611
0.98,0.4736,0.431,0.4135,0.4043,0.4105,0.4029,0.4086,0.4113,0.4127,0.4182,0.4326,0.4157,0.4201,0.4222,0.4281


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0404,-0.0378,-0.0379,-0.0393,-0.041,-0.0422,-0.0424,-0.0433,-0.0425,-0.0431,-0.0438,-0.0438,-0.0429,-0.0447,-0.0436
0.05,-0.0347,-0.0317,-0.0318,-0.0328,-0.0342,-0.0354,-0.0359,-0.0362,-0.0361,-0.0359,-0.0371,-0.037,-0.0371,-0.0373,-0.0366
0.25,-0.0209,-0.0168,-0.0167,-0.0175,-0.0183,-0.0185,-0.0197,-0.0198,-0.0203,-0.0201,-0.0206,-0.0206,-0.021,-0.0207,-0.021
0.5,-0.011,-0.0063,-0.0059,-0.0066,-0.0067,-0.0074,-0.0085,-0.0088,-0.0089,-0.0089,-0.01,-0.0097,-0.0097,-0.0094,-0.0099
0.75,-0.0012,0.004,0.0046,0.0045,0.0044,0.0034,0.0028,0.0024,0.0021,0.0019,0.0012,0.0016,0.0016,0.0018,0.0014
0.95,0.0127,0.019,0.0208,0.0197,0.0203,0.0194,0.019,0.0187,0.0184,0.0182,0.0172,0.0177,0.0173,0.0175,0.017
0.98,0.0185,0.0255,0.0275,0.0259,0.0263,0.0258,0.0258,0.0254,0.0249,0.0248,0.0242,0.0248,0.0241,0.0242,0.0235


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0043,-0.0058,-0.0047,-0.0037,-0.0032,-0.0028,-0.0022,-0.002,-0.0021,-0.0019,-0.002,-0.0012,-0.0013,-0.0017,-0.0013
0.05,-0.0019,-0.0023,-0.0007,0.0001,0.001,0.0015,0.002,0.0021,0.0018,0.002,0.0023,0.0025,0.0026,0.0024,0.0025
0.25,0.0038,0.0064,0.0085,0.0098,0.0107,0.0112,0.0115,0.0118,0.012,0.0121,0.0122,0.0124,0.0122,0.0122,0.0122
0.5,0.008,0.0123,0.0149,0.0165,0.0173,0.018,0.0182,0.0185,0.0187,0.0189,0.0191,0.0191,0.0191,0.0192,0.019
0.75,0.012,0.0184,0.0214,0.0231,0.0242,0.0247,0.025,0.0254,0.0252,0.0255,0.0257,0.0259,0.0259,0.026,0.0257
0.95,0.0182,0.0268,0.0307,0.0325,0.0338,0.0346,0.0349,0.0352,0.0351,0.035,0.0356,0.0356,0.0356,0.0357,0.0357
0.98,0.0206,0.0308,0.0348,0.0365,0.038,0.0389,0.0391,0.0391,0.0394,0.0395,0.0396,0.04,0.0396,0.0397,0.0398


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0031,-0.0089,-0.0108,-0.0114,-0.0122,-0.0124,-0.0124,-0.0125,-0.0128,-0.0126,-0.0127,-0.0136,-0.0127,-0.0132,-0.0131
0.05,0.0033,-0.0019,-0.004,-0.0045,-0.0056,-0.0056,-0.0056,-0.0056,-0.0066,-0.006,-0.0061,-0.0068,-0.0063,-0.0066,-0.0068
0.25,0.0181,0.0133,0.0116,0.0113,0.0099,0.0098,0.0101,0.0098,0.0095,0.0093,0.0102,0.0093,0.0093,0.0092,0.0098
0.5,0.0286,0.0243,0.0228,0.0221,0.0213,0.021,0.0211,0.0207,0.0206,0.0202,0.0208,0.0204,0.0205,0.02,0.0205
0.75,0.0392,0.035,0.0338,0.0329,0.0324,0.0318,0.0321,0.0317,0.0319,0.0312,0.0319,0.031,0.0311,0.0311,0.0314
0.95,0.0538,0.0505,0.0496,0.0483,0.0485,0.0479,0.0477,0.0479,0.0474,0.0463,0.0472,0.0471,0.0467,0.0471,0.0465
0.98,0.06,0.0569,0.055,0.0544,0.0552,0.0551,0.0537,0.0544,0.0538,0.0528,0.0535,0.0544,0.0534,0.0531,0.0532


In [167]:
scenarios_b = simulate(bmodel,freq * 15, 10)

for n in 1:4
    print_percentiles(scenarios_b[n, :, 2:end], [0.05, 0.25, 0.5, 0.75, 0.95], freq, string(assets_names[n]))
 end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.2534,-0.7706,-0.8996,-0.9366,-0.9616,-0.9797,-0.988,-0.9934,-1.011,-1.0162,-1.0222,-1.0206,-1.0291,-1.033,-1.0279
0.25,0.0231,-0.2032,-0.2635,-0.2809,-0.2911,-0.2961,-0.2968,-0.2984,-0.3023,-0.3001,-0.3009,-0.3049,-0.3031,-0.3023,-0.3014
0.5,0.1783,0.1056,0.0832,0.0762,0.0759,0.0768,0.0801,0.0761,0.0763,0.0781,0.0816,0.0818,0.0848,0.083,0.085
0.75,0.3517,0.4221,0.4337,0.4357,0.4464,0.4503,0.4568,0.4601,0.4585,0.4651,0.4672,0.4717,0.4716,0.4739,0.4757
0.95,0.6701,1.0011,1.0808,1.1015,1.1244,1.1466,1.1543,1.1683,1.1714,1.1902,1.1967,1.2048,1.2105,1.2112,1.2259


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0527,-0.0779,-0.0888,-0.0954,-0.1005,-0.105,-0.1089,-0.1118,-0.114,-0.1158,-0.1177,-0.1193,-0.1206,-0.122,-0.124
0.25,-0.0256,-0.0312,-0.0347,-0.037,-0.0392,-0.0408,-0.0418,-0.0428,-0.0436,-0.0442,-0.0446,-0.0449,-0.0453,-0.0457,-0.0462
0.5,-0.0104,-0.0056,-0.0053,-0.0057,-0.0061,-0.0068,-0.0075,-0.0077,-0.008,-0.0083,-0.0085,-0.0086,-0.0088,-0.0091,-0.0092
0.75,0.0044,0.0202,0.0241,0.026,0.0267,0.0272,0.0273,0.0273,0.0276,0.0278,0.0274,0.0275,0.0273,0.0277,0.0274
0.95,0.0288,0.0668,0.079,0.0844,0.0887,0.0921,0.0948,0.0965,0.0988,0.1003,0.1031,0.1036,0.1047,0.1065,0.1068


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0289,-0.0451,-0.0524,-0.0579,-0.0618,-0.065,-0.0674,-0.069,-0.0708,-0.0723,-0.0741,-0.0758,-0.0767,-0.0778,-0.0788
0.25,-0.0066,-0.0093,-0.0098,-0.0103,-0.0103,-0.0106,-0.0105,-0.0104,-0.0102,-0.0104,-0.0104,-0.0107,-0.0108,-0.0104,-0.0103
0.5,0.0079,0.0122,0.0145,0.0159,0.0168,0.0173,0.0177,0.0181,0.0184,0.0183,0.0185,0.0187,0.0189,0.019,0.0192
0.75,0.0222,0.033,0.0384,0.0413,0.0433,0.0446,0.0454,0.0462,0.0467,0.047,0.047,0.0477,0.0481,0.0481,0.0484
0.95,0.0439,0.0659,0.0772,0.0849,0.0894,0.0939,0.0969,0.0994,0.1023,0.1037,0.1051,0.1069,0.1084,0.1096,0.1103


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0013,-0.0204,-0.0282,-0.0321,-0.035,-0.0373,-0.0392,-0.0405,-0.0417,-0.0434,-0.0439,-0.0448,-0.0465,-0.0469,-0.0484
0.25,0.0181,0.0082,0.0055,0.0038,0.003,0.0022,0.0018,0.0016,0.0011,0.001,0.0008,0.0007,0.0005,0.0005,0.0003
0.5,0.0289,0.0235,0.0224,0.0216,0.0214,0.0211,0.0211,0.0209,0.0207,0.0207,0.0206,0.0206,0.0205,0.0204,0.0204
0.75,0.0397,0.0392,0.0397,0.0398,0.0402,0.0401,0.0403,0.0404,0.0404,0.0405,0.0404,0.0406,0.0404,0.0406,0.0404
0.95,0.0584,0.0695,0.0743,0.0773,0.0796,0.0809,0.0826,0.0834,0.0844,0.086,0.0864,0.0875,0.0886,0.0885,0.0888


In [168]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[1 * freq, freq * 5, freq * 10, freq * 15]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], 1, string(assets_names[n]))    
end

,1,2,3,4
0.02,-0.0933,-0.0226,0.0039,0.0165
0.1,0.0124,0.027,0.0378,0.0459
0.25,0.0977,0.0665,0.0655,0.0688
0.5,0.2069,0.1119,0.0972,0.0946
0.75,0.3278,0.1599,0.13,0.1216
0.9,0.4487,0.2053,0.1609,0.1469
0.98,0.6058,0.2631,0.1998,0.1772


,1,2,3,4
0.02,-0.0396,-0.0221,-0.0206,-0.0198
0.1,-0.0292,-0.0166,-0.0157,-0.0154
0.25,-0.0206,-0.0122,-0.012,-0.0121
0.5,-0.011,-0.0073,-0.0079,-0.0085
0.75,-0.0012,-0.0025,-0.0038,-0.0048
0.9,0.0073,0.002,-0.0001,-0.0015
0.98,0.0187,0.0079,0.0046,0.0028


,1,2,3,4
0.02,-0.0043,-0.0002,0.0039,0.0062
0.1,0.0003,0.0051,0.0085,0.0104
0.25,0.0038,0.0092,0.0121,0.0137
0.5,0.0081,0.0139,0.0163,0.0173
0.75,0.0121,0.0186,0.0204,0.0209
0.9,0.016,0.0228,0.024,0.0242
0.98,0.0208,0.0282,0.0289,0.0283


In [169]:
collect(.05:.1:.95)

10-element Vector{Float64}:
 0.05
 0.15
 0.25
 0.35
 0.45
 0.55
 0.65
 0.75
 0.85
 0.95

In [170]:
for n in 1:3
    print_percentiles(scenarios[n, :, 2:end],[.05, .15, .25, .35, .45, .55, .65, 0.75, .85, 0.95], freq*5, string(assets_names[n]))
end


,1,2,3
0.05,0.0155,-0.1745,-0.1622
0.15,0.2052,0.0298,0.0611
0.25,0.322,0.1611,0.1895
0.35,0.41,0.265,0.294
0.45,0.4912,0.3583,0.3881
0.55,0.5721,0.4417,0.4745
0.65,0.6569,0.5329,0.5702
0.75,0.7416,0.6319,0.6763
0.85,0.8573,0.7628,0.809
0.95,1.0391,0.9735,1.0137


,1,2,3
0.05,-0.0968,-0.1154,-0.1213
0.15,-0.0738,-0.0885,-0.094
0.25,-0.0612,-0.072,-0.0776
0.35,-0.0508,-0.0591,-0.0652
0.45,-0.0411,-0.0478,-0.0536
0.55,-0.0323,-0.0369,-0.043
0.65,-0.0226,-0.0261,-0.0319
0.75,-0.0126,-0.0137,-0.0186
0.85,0.0004,0.0021,-0.0028
0.95,0.0235,0.0289,0.0234


,1,2,3
0.05,0.0138,0.0282,0.0303
0.15,0.0334,0.0519,0.0545
0.25,0.0459,0.0656,0.069
0.35,0.0563,0.0772,0.0802
0.45,0.0652,0.0872,0.0899
0.55,0.0738,0.0973,0.1002
0.65,0.0824,0.1071,0.1107
0.75,0.0923,0.1186,0.1218
0.85,0.1043,0.1324,0.1358
0.95,0.1243,0.1562,0.1593


In [171]:
 cov_m = cov(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI])) 
 mu = mean(values(s_data[:Total_Stock_T_Bill, :Price_10Y_T_Bill, :T_Bill_CPI]), dims=1) 

KeyError: KeyError: key :Price_10Y_T_Bill not found

In [172]:
using Distributions
scenarios_n = zeros(3,100_000,freq*15)

for t in 1:freq*15
    scenarios_n[:,:,t] = rand(MvNormal(vec(mu), cov_m), 100_000)
end



UndefVarError: UndefVarError: `mu` not defined

In [173]:
for n in 1:3
    print_percentiles(scenarios_n[n, :, 1:end],[0.02, 0.25, 0.5, 0.75, 0.98], 12, string(assets_names[n]))
end

,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


,1,2,3,4,5
0.02,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0
0.5,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0
0.98,0.0,0.0,0.0,0.0,0.0


In [174]:
returns_summarystats(data_source[:Yeld_10Y],1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Yeld_10Y,0.0292,0.1584,0.0548,0.0736,0.0066,0.9377,0.0333,0.9774,0.6945,1.8772,0.0468


In [175]:
yelds10y = dropdims(sum(scenarios[[3,5,6],:,:], dims=1), dims=1)
print_percentiles(annualise(yelds10y,4), [0.02, 0.25, 0.5, 0.75, 0.98])

BoundsError: BoundsError: attempt to access 4×10000×61 Array{Float64, 3} at index [[3, 5, 6], 1:10000, 1:61]

In [176]:
include("generate_bond_temp.jl")

using .Bonds

bond_returns = Bonds.scenarios_bond_returns(yelds10y, 10, freq)
print_percentiles(bond_returns, [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `yelds10y` not defined

In [177]:
#cpi_ann = annualise(scenarios[6,:,2:end],4)
t_bill = scenarios[3,:,2:end]
excess_bond_returns = bond_returns .- t_bill
print_percentiles(expm1.(excess_bond_returns), [0.02, 0.25, 0.5, 0.75, 0.98], freq)

UndefVarError: UndefVarError: `bond_returns` not defined

In [178]:
scenario = 3
plot(yelds10y[scenario,:], label="yeld")
plot!(twinx(), bond_returns[scenario,:], label="bond returns", color=:orange)

UndefVarError: UndefVarError: `yelds10y` not defined

In [179]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.18171351596139262
 -0.027232400947970963
  0.08072507572167638
  0.18839880420174726
  0.3424924311028206

In [180]:
sim = rand(Normal(.08, .16), 100_000)
quantile(sim, [.05, .25, .5, .75, .95])

5-element Vector{Float64}:
 -0.182365998591205
 -0.02788927536093948
  0.0794699326302567
  0.18675325407315668
  0.34193186241516266

In [181]:
simt = rand(LocationScale(.04, .075, TDist(2.5)), 100_000)
quantile(simt, [.02, .25, .5, .75, .98])

5-element Vector{Float64}:
 -0.2541433959092337
 -0.018505769827709705
  0.04039838853185285
  0.09923819478905441
  0.33912179049623004